In [12]:
from fenics import *
from dolfin import *

import numpy as np
import matplotlib.pyplot as plt
import h5py

import pdb

class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        # return bool(((x[0] < DOLFIN_EPS) or (x[0] > 1 + DOLFIN_EPS)) and on_boundary)
        return bool((x[0] < DOLFIN_EPS) or (x[0] > (1.0 + DOLFIN_EPS)))# and on_boundary)
        # return bool(- DOLFIN_EPS < x[0] < DOLFIN_EPS and on_boundary)

    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        y[0] = x[0] - 1

# Create periodic boundary condition
t_res = 100024#50000
x_res = 128
x_coordinate = np.linspace(0,1, x_res)
t_coordinate = np.linspace(0,1, t_res)

for _nu_ in [1e-4]:#[1e-4,1e-3,1e-2,1e-1]:
    i = 0
    solution = np.zeros((30,t_res,x_res))
    # pdb.set_trace()
    Amp = [np.random.uniform(), np.random.uniform()]
    k = np.random.randint(8, size=2) * 2 * np.pi
    phi = [np.random.uniform(0, 2 * np.pi), np.random.uniform(0, 2 * np.pi)]
    # Create periodic boundary condition
    pbc = PeriodicBoundary()

    n = x_res
    mesh = UnitIntervalMesh(n)
    V = VectorFunctionSpace(mesh, "P", 1, constrained_domain = pbc) # setting up the space/mesh

    initial_cond = "{}*sin({}*x[0] + {}) + {}*sin({}*x[0] + {})".format(Amp[0], k[0], phi[0],Amp[1], k[1], phi[1])  ## defnining initial cond
    u = project(Expression((initial_cond,), degree = 1),  V)

    u_next = Function(V) 
    v = TestFunction(V)

    nu = Constant(_nu_) / np.pi # constant definintion 

    timestep = Constant(t_coordinate[1])
    F = (inner((u_next - u)/timestep, v) + inner(grad(u_next)*u_next, v)+ nu*inner(grad(u_next), grad(v)))*dx
    ### ^^^^^ definind non linear variational problem

    # u0 = Constant(0.0)
    # dbc = DirichletBoundary()
    # bc = DirichletBC(V, [u0], dbc)

    t = t_coordinate[1]#0.01
    end = max(t_coordinate)
    j = 1
    solution[i,0,:] =  u.vector()[:]
    while (t <= end):

        # solve(F == 0, u_next, [], 
        #     solver_parameters={"newton_solver":{"relative_tolerance":1e-3,
        #                                         "absolute_tolerance":1e-6}}) ## solving non linear problem
        solve(F == 0, u_next, []) ## solving non linear problem


        u.assign(u_next)
        t += float(timestep)
        solution[i,j,:] = u.vector()[:]
        j += 1

    f = h5py.File('/home/divyam123/fenics/burgers/1D_Burgers_Sols_Nu{}.hdf5'.format(_nu_), 'w')
    f['x-coordinate'] = x_coordinate
    f['t-coordinate'] = t_coordinate
    f['tensor'] = solution
    f.close()



In [11]:
j

10024